In [367]:
!pip install vaderSentiment

In [368]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [369]:
#import Library
import pandas as pd
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from collections import Counter

In [0]:
nlp = spacy.load('en_core_web_lg')


In [372]:
import requests
url = 'https://raw.githubusercontent.com/Moly-malibu/Tag-Smart-Emails/master/Enronemails.csv?token=AN2RW2GGHXCN4RRW3NT6C2S65KRHS'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

24004915

In [0]:
df = pd.read_csv("email.csv").drop(['Unnamed: 0'], axis=1)

In [374]:
[s.strip() for s in df['message'][100].splitlines()]

['Message-ID: <15940494.1075855689309.JavaMail.evans@thyme>',
 'Date: Wed, 16 Aug 2000 05:35:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: cooper.richey@enron.com',
 'Subject: Re:',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: Cooper Richey',
 'X-cc:',
 'X-bcc:',
 "X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen.nsf',
 '',
 "I tried the new address but I don't have access.  also, what do I need to",
 'enter under domain?']

In [0]:
emails = ' '.join([s.strip() for s in df['message'][0].splitlines()][15:])

In [0]:
dates = ' '.join([s.strip() for s in df['message'][1].splitlines()][1:2])

In [0]:
from_who = ' '.join([s.strip('') for s in df['message'][1].splitlines()][2:3])

In [0]:
to_who = ' '.join([s.strip('') for s in df['message'][1].splitlines()][3:4])

In [0]:
subject = ' '.join([s.strip('') for s in df['message'][1].splitlines()][4:5])

In [0]:
content_type = ' '.join([s.strip('') for s in df['message'][1].splitlines()][6:7])

In [0]:
#CLEAN DATASET
import re
import html
def cleanup_html(raw_html):
    clean_html = re.sub(r'<.*?>', '', raw_html)
    clean_html_http = re.sub(r'http\S+([\.]{3})?', '', clean_html)
    clean_txt = html.unescape(clean_html)
    return clean_txt
df['email_body'] = df['message'].apply(cleanup_html)

In [382]:
def clean_email(email):
    return ' '.join([s.strip() for s in email.splitlines()][15:])
def clean_content_type(email):
    return ' '.join([s.strip() for s in email.splitlines()][6:7])
def clean_subject(email):
    return ' '.join([s.strip() for s in email.splitlines()][4:5])
def clean_to_who(email):
    return ' '.join([s.strip() for s in email.splitlines()][4:4])
def clean_from_who(email):
    return ' '.join([s.strip() for s in email.splitlines()][2:3])
def clean_dates(email):
    return ' '.join([s.strip() for s in email.splitlines()][1:2])

df['email_body'] = df['message'].apply(clean_email)
df['subject'] = df['message'].apply(clean_subject)
df['content_type'] = df['message'].apply(clean_content_type)
df['to_who'] = df['message'].apply(clean_to_who)
df['from_who'] = df['message'].apply(clean_from_who)
df['dates'] = df['message'].apply(clean_dates)
df["email_body"].value_counts(sort=True).head()

 most bullish thing at this point is moving closer to everyone's psychological $4 price target and that everybody and their dog is still short.  next sellers need to be from producer community.  saw a little this week with williams hedging the barrett transaction but wouldnt say thats indicative of the rest of the e&p community.  short covering rallies will get more common here.  velocity of move down has slowed significantly for good (except maybe in bid week).  my concern is if we go to $4 and people want to cover some shorts, who's selling it to them?  might feel a lot like it did when we were trying to break $5.    9
 Jim,  Is there going to be a conference call or some type of weekly meeting about all the regulatory issues facing California this week?  Can you make sure the gas desk is included.  Phillip                                                                                                                                                                                    

In [383]:
df.shape

(10000, 8)

In [0]:
#CLEAN DATA

#Str the data
df['email_body'] = df['email_body'].astype(str)

#remove whitespace
df['email_body'] = df['email_body'].str.strip().str.lower()

#Start with date
df['email_body'].str.match('\d?\d/\d?\d/\d{4}').all()

#\s indicates a white space. So [^\s] is any non-white space and includes letters, numbers, special characters
df['email_body'] = df['email_body'].str.replace('[^a-zA-Z\s]', '').str.replace('\s+', ' ')

In [385]:
df.shape

(10000, 8)

In [0]:
#Vander model
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 

def vaderizeSentiment(sentence):
    return analyzer.polarity_scores(sentence)

In [0]:
#create columns score with numbers
df['Scores'] = df['email_body'].apply(vaderizeSentiment)

In [388]:
df.shape

(10000, 9)

In [0]:
df[['negative', 'neutral', 'positive', 'compound']] = df.Scores.apply(pd.Series)

In [0]:
#Model to see final prediction
Model = SentimentIntensityAnalyzer()
def score(sentence, Model):
    return Model.polarity_scores(sentence)['compound']

# Sentiment score
df['final_score'] = df['email_body'].apply(lambda x: score(x, Model))

#Discretize variable.
df['final_pred'] = pd.cut(df['final_score'], bins=5, labels=[1, 2, 3, 4, 5])
#df = df.drop('final_score', axis=1).sort_values(by=['file','final_pred']).iloc[:5]

In [391]:
df.shape

(10000, 15)

In [0]:
tokens = []
for doc in nlp.pipe(df['email_body'], batch_size=500):

    doc_tokens = []

    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.lemma_.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens


In [0]:
df = df.drop(columns = ['file', 'message', 'Scores'])

In [394]:
df.shape

(10000, 13)

In [416]:
url = 'https://raw.githubusercontent.com/Lambda-School-Labs/tagger-ds/master/notebooks/tags.csv'
myfile = requests.get(url)
open('tags.csv', 'wb').write(myfile.content)

13296306

In [0]:
df_tags = pd.read_csv("tags.csv")

In [0]:
df_tags.rename(columns={'tags':'smart_tags'}, inplace=True)

In [0]:
frames = [df, df_tags]

result = pd.concat([df, df_tags], axis=1, sort=False)

In [0]:
result = result.drop(columns=['email_body', 'tokens', 'final_score'])

In [422]:
result

,subject,content_type,to_who,from_who,dates,negative,neutral,positive,compound,final_pred,body_email,smart_tags,negative,neutral,positive,compound,final_pred
0,Subject:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,here is our forecast,['forecast'],0.000,1.000,0.000,0.0000,3.0
1,Subject: Re:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)",0.000,0.893,0.107,0.9313,5,traveling to have a business meeting takes the...,"['travel', 'business', 'meeting', 'take', 'fun...",0.000,0.893,0.107,0.9313,5.0
2,Subject: Re: test,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",0.000,0.513,0.487,0.5859,4,test successful way to go,"['test', 'successful', 'way']",0.000,0.513,0.487,0.5859,4.0
3,Subject:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,randy can you send me a schedule of the salary...,"['randy', 'send', 'schedule', 'salary', 'level...",0.000,1.000,0.000,0.0000,3.0
4,Subject: Re: Hello,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",0.375,0.625,0.000,-0.3400,2,lets shoot for tuesday at,"['let', 'shoot', 'tuesday']",0.375,0.625,0.000,-0.3400,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Subject: Re: Evite: Super Bowl Party,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:31:00 -0800 (PST)",0.031,0.799,0.170,0.9895,5,NaN,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ...",NaN,NaN,NaN,NaN,NaN
9996,Subject:,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:15:00 -0800 (PST)",0.262,0.738,0.000,-0.4939,2,NaN,"['copy', 'idea', 'screw']",NaN,NaN,NaN,NaN,NaN
9997,Subject: 9912 Texas Financial Liquidations,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 01:36:00 -0800 (PST)",0.000,0.914,0.086,0.4404,4,NaN,"[' ', 'forward', 'eric', 'basshouect', 'troy',...",NaN,NaN,NaN,NaN,NaN
9998,Subject:,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Wed, 6 Dec 2000 06:03:00 -0800 (PST)",0.000,1.000,0.000,0.0000,3,NaN,"['buy', 'enron', 's']",NaN,NaN,NaN,NaN,NaN


In [0]:
result['message_id'] = result.index + 1

In [424]:
result

,subject,content_type,to_who,from_who,dates,negative,neutral,positive,compound,final_pred,body_email,smart_tags,negative,neutral,positive,compound,final_pred,message_id
0,Subject:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,here is our forecast,['forecast'],0.000,1.000,0.000,0.0000,3.0,1
1,Subject: Re:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)",0.000,0.893,0.107,0.9313,5,traveling to have a business meeting takes the...,"['travel', 'business', 'meeting', 'take', 'fun...",0.000,0.893,0.107,0.9313,5.0,2
2,Subject: Re: test,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",0.000,0.513,0.487,0.5859,4,test successful way to go,"['test', 'successful', 'way']",0.000,0.513,0.487,0.5859,4.0,3
3,Subject:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,randy can you send me a schedule of the salary...,"['randy', 'send', 'schedule', 'salary', 'level...",0.000,1.000,0.000,0.0000,3.0,4
4,Subject: Re: Hello,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",0.375,0.625,0.000,-0.3400,2,lets shoot for tuesday at,"['let', 'shoot', 'tuesday']",0.375,0.625,0.000,-0.3400,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Subject: Re: Evite: Super Bowl Party,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:31:00 -0800 (PST)",0.031,0.799,0.170,0.9895,5,NaN,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ...",NaN,NaN,NaN,NaN,NaN,9996
9996,Subject:,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:15:00 -0800 (PST)",0.262,0.738,0.000,-0.4939,2,NaN,"['copy', 'idea', 'screw']",NaN,NaN,NaN,NaN,NaN,9997
9997,Subject: 9912 Texas Financial Liquidations,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 01:36:00 -0800 (PST)",0.000,0.914,0.086,0.4404,4,NaN,"[' ', 'forward', 'eric', 'basshouect', 'troy',...",NaN,NaN,NaN,NaN,NaN,9998
9998,Subject:,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Wed, 6 Dec 2000 06:03:00 -0800 (PST)",0.000,1.000,0.000,0.0000,3,NaN,"['buy', 'enron', 's']",NaN,NaN,NaN,NaN,NaN,9999


In [0]:
result['user_tags'] = np.nan
result['thread_id'] = np.nan

In [427]:
result

,subject,content_type,to_who,from_who,dates,negative,neutral,positive,compound,final_pred,body_email,smart_tags,negative,neutral,positive,compound,final_pred,message_id,user_tags,thread_id
0,Subject:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,here is our forecast,['forecast'],0.000,1.000,0.000,0.0000,3.0,1,NaN,NaN
1,Subject: Re:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)",0.000,0.893,0.107,0.9313,5,traveling to have a business meeting takes the...,"['travel', 'business', 'meeting', 'take', 'fun...",0.000,0.893,0.107,0.9313,5.0,2,NaN,NaN
2,Subject: Re: test,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",0.000,0.513,0.487,0.5859,4,test successful way to go,"['test', 'successful', 'way']",0.000,0.513,0.487,0.5859,4.0,3,NaN,NaN
3,Subject:,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,randy can you send me a schedule of the salary...,"['randy', 'send', 'schedule', 'salary', 'level...",0.000,1.000,0.000,0.0000,3.0,4,NaN,NaN
4,Subject: Re: Hello,Content-Type: text/plain; charset=us-ascii,,From: phillip.allen@enron.com,"Date: Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",0.375,0.625,0.000,-0.3400,2,lets shoot for tuesday at,"['let', 'shoot', 'tuesday']",0.375,0.625,0.000,-0.3400,2.0,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Subject: Re: Evite: Super Bowl Party,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:31:00 -0800 (PST)",0.031,0.799,0.170,0.9895,5,NaN,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ...",NaN,NaN,NaN,NaN,NaN,9996,NaN,NaN
9996,Subject:,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:15:00 -0800 (PST)",0.262,0.738,0.000,-0.4939,2,NaN,"['copy', 'idea', 'screw']",NaN,NaN,NaN,NaN,NaN,9997,NaN,NaN
9997,Subject: 9912 Texas Financial Liquidations,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 01:36:00 -0800 (PST)",0.000,0.914,0.086,0.4404,4,NaN,"[' ', 'forward', 'eric', 'basshouect', 'troy',...",NaN,NaN,NaN,NaN,NaN,9998,NaN,NaN
9998,Subject:,Content-Type: text/plain; charset=us-ascii,,From: eric.bass@enron.com,"Date: Wed, 6 Dec 2000 06:03:00 -0800 (PST)",0.000,1.000,0.000,0.0000,3,NaN,"['buy', 'enron', 's']",NaN,NaN,NaN,NaN,NaN,9999,NaN,NaN


In [0]:
def save_data_frame_as_csv(result=None, file=None):
    """
    Save a Pandas DataFrame to CSV format.
    """
    try:
        if not file.endswith('.csv'):
            file += '.csv'
        result.to_csv(file, index=False)
        print(f'DataFrame saved as: {file}')
    except Exception as e:
        print("DataFrame couldn't be saved: ", sys.exc_info()[0])
        raise

In [429]:
save_data_frame_as_csv(result, './merged_data_for_KNN.csv')

DataFrame saved as: ./merged_data_for_KNN.csv
